# Geospatial Processing
## 1. Contexte
- Source trips : iceberg.silver.trips_complete
- Source zones : iceberg.bronze.taxi_zones
- Output : iceberg.silver.trips_geospatial
- Librairies : GeoPandas (offline) + Spark

## 2. Imports & setup

In [27]:
import geopandas as gpd
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType
from pyspark.sql.window import Window
from math import radians, sin, cos, sqrt, atan2


## 3. Chargement et enrichissement des zones (GeoPandas)
- On calcule les centroïdes et la surface
- Une seule fois → stocké en Silver

In [33]:
def create_zones_enhanced():
    print("🗺️  Enrichissement géospatial des zones taxi")

    # Lecture shapefile
    gdf = gpd.read_file("/tmp/taxi_zones/taxi_zones.shp")

    # 1️⃣ Reprojection vers CRS projeté (NY State Plane - feet)
    gdf_proj = gdf.to_crs(epsg=2263)

    # Centroïdes (corrects)
    gdf_proj["centroid"] = gdf_proj.geometry.centroid

    # Surface en miles²
    gdf_proj["area_sq_miles"] = gdf_proj.geometry.area / (5280 ** 2)

    # 2️⃣ Retour en GPS
    gdf_proj = gdf_proj.to_crs(epsg=4326)

    gdf_proj["centroid_lon"] = gdf_proj.centroid.x
    gdf_proj["centroid_lat"] = gdf_proj.centroid.y

    zones_df = spark.createDataFrame(
        gdf_proj[[
            "LocationID",
            "zone",
            "borough",
            "centroid_lat",
            "centroid_lon",
            "area_sq_miles"
        ]]
    )

    zones_df.writeTo("iceberg.silver.zones_enhanced") \
            .createOrReplace()

    print("✅ silver.zones_enhanced créée")

    return zones_df


In [34]:
from pyspark.sql.functions import udf

@udf(returnType=DoubleType())
def haversine_distance(lat1, lon1, lat2, lon2):
    if None in (lat1, lon1, lat2, lon2):
        return None

    R = 3958.8  # miles

    lat1, lon1, lat2, lon2 = map(
        radians, [lat1, lon1, lat2, lon2]
    )

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c


In [35]:
def create_trips_geospatial():
    print("🚕 Enrichissement géospatial des trips")

    trips = spark.table("iceberg.silver.trips_complete")
    zones = spark.table("iceberg.silver.zones_enhanced")

    trips_geo = trips.alias("t") \
        .join(
            zones.alias("pz"),
            F.col("t.PULocationID") == F.col("pz.LocationID"),
            "left"
        ) \
        .select(
            "t.*",
            F.col("pz.centroid_lat").alias("pickup_lat"),
            F.col("pz.centroid_lon").alias("pickup_lon"),
            F.col("pz.area_sq_miles").alias("pickup_area")
        )

    trips_geo = trips_geo.alias("t") \
        .join(
            zones.alias("dz"),
            F.col("t.DOLocationID") == F.col("dz.LocationID"),
            "left"
        ) \
        .select(
            "t.*",
            F.col("dz.centroid_lat").alias("dropoff_lat"),
            F.col("dz.centroid_lon").alias("dropoff_lon"),
            F.col("dz.area_sq_miles").alias("dropoff_area")
        )

    trips_geo = trips_geo.withColumn(
        "real_distance_miles",
        haversine_distance(
            "pickup_lat", "pickup_lon",
            "dropoff_lat", "dropoff_lon"
        )
    )

    trips_geo = trips_geo.withColumn(
        "distance_error_ratio",
        F.when(
            F.col("trip_distance") > 0,
            F.abs(F.col("real_distance_miles") - F.col("trip_distance"))
            / F.col("trip_distance")
        )
    )

    trips_geo.writeTo("iceberg.silver.trips_geospatial") \
              .partitionedBy("year", "month") \
              .createOrReplace()

    print("✅ silver.trips_geospatial créée")

    return trips_geo


In [36]:
zones_enhanced = create_zones_enhanced()
trips_geospatial = create_trips_geospatial()


🗺️  Enrichissement géospatial des zones taxi


/tmp/ipykernel_377/3086974347.py:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_proj["centroid_lon"] = gdf_proj.centroid.x
/tmp/ipykernel_377/3086974347.py:20: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_proj["centroid_lat"] = gdf_proj.centroid.y


✅ silver.zones_enhanced créée
🚕 Enrichissement géospatial des trips


AnalysisException: Couldn't find column year in:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- trip_duration_minutes: double (nullable = true)
 |-- speed_mph: double (

In [ ]:
spark.table("iceberg.silver.trips_geospatial") \
     .select(
         "trip_distance",
         "real_distance_miles",
         "distance_error_ratio"
     ) \
     .show(10, truncate=False)
